# explorations into a dummy model

### dummy performance input

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.options.mode.chained_assignment = None

In [ ]:
# time input
stamps = pd.date_range('2021-01-01','2021-03-01',freq='D')
dates = [i.strftime("%Y-%m-%d") for i in stamps]

In [ ]:
# simulating turnover
n = len(np.array(dates))
x = np.arange(-n/2,n/2,1,dtype=np.float64)
m = np.random.uniform(0.35,0.4,(n,))
b = np.random.uniform(5,10,(n,))
turnover = x * m + b

# simulating mkt expenses
x = np.arange(-n/2,n/2,1,dtype=np.float64)
m = np.random.uniform(0.1,0.4,(n,))
b = np.random.uniform(1,2,(n,))
mkt_cost = x * m + b

In [ ]:
# load data into pandas dataframe
df = pd.DataFrame()
df['date'] = dates
df['turnover'] = turnover
df['mkt_cost'] = mkt_cost

In [ ]:
# visual inspection
df.plot()

### naive prediction on future performance

Reference to facebook prophet: https://facebook.github.io/prophet/docs/quick_start.html

In [ ]:
from fbprophet import Prophet

# prepare data
data = df[['date','turnover']]
data.columns = ['ds','y']

# make a prediction with Prophet
m = Prophet()
m.fit(data)
future = m.make_future_dataframe(periods=15)
forecast = m.predict(future)
#forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)

### target group input

In [ ]:
# geography input
filepath = "../data/processed/gv100ad.csv"
geo = pd.read_csv(filepath)

# explore dataset for larger area around Aschaffenburg
geo.loc[(geo.rb == "Reg.-Bez. Darmstadt") | (geo.rb == "Unterfranken")].groupby("kreis").agg(
    {
        'gemeindename': 'count',
        'bev_geseamt': 'sum',
        'bev_maennl': 'sum',
        'bev_weibl': 'sum',
    }
).sort_values("bev_geseamt", ascending=False).head(10)

In [ ]:
# make a selection for targeted area
selected_districts = ['Aschaffenburg','Offenbach','Miltenberg','Darmstadt-Dieburg']
target_market = geo.loc[geo['kreis'].isin(selected_districts)]

population = target_market['bev_geseamt'].sum()
communities = target_market['gemeindename'].unique()

print(f"Wow! Dein Zielmarkt hat eine Gesamtpopulation von {population} Einwohnern verteilt auf {len(communities)} Gemeinden.")

In [ ]:
# set a target selection for the total population of your market
target_share = 0.05
target_market['target_pop'] = target_market["bev_geseamt"] * target_share
target_pop = target_market['target_pop'].sum()

print(f"{int(target_share*100)}% aller Personen fallen in deine Zielgruppe. Das sind immer noch {int(target_pop)} potentielle Zielkunden. Amazing!")

In [ ]:
# explore top communities
target_market.groupby(['kreis','vb']).agg(
    {
        'gemeindename': 'count',
        'bev_geseamt': 'sum',
        'bev_maennl': 'sum',
        'bev_weibl': 'sum',
        'target_pop': 'sum',
    }
).sort_values("bev_geseamt", ascending=False).head(5)

In [ ]:
# validity check with home community
home_community = ["Heimbuchenthal", "Mespelbrunn"]
target_valid = target_market[target_market['gemeindename'].isin(home_community)]['target_pop'].sum()
pop_valid = target_market[target_market['gemeindename'].isin(home_community)]['bev_geseamt'].sum()

print(f"{len(home_community)} Validierungsgemeinde(n) in der Auswahl: {', '.join(home_community)}.")
print(f"> Hier gibt es eine Gesamtbevolkerung von {int(pop_valid)} Personen (Angabe von stat. Bundesamt, Dez. 2019).")
print(f"> Grundlage deiner Angaben({int(target_share*100)}%-Zielkundenanteil), schätzen wir eine Anzahl von etwa {int(target_valid)} Personen als Zielkunden.")
print("\nAchtung!1!! Jetzt wieder aufwachen. Preisfrage:")
print(f"> Kannst du dir vorstellen, dass {int(target_valid)} Personen in deiner Hood dein Zeug kaufen wollen?")
print("> Wenn deine Antwort 'Nein' lauten sollte, dann sitz dir halt net den ganzen Tag den Arsch platt sondern änder bitte was an deinen Annahmen.")